# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [2]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('C:/Users/huiho/production/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [3]:
from sklearn.model_selection import train_test_split

# Create the features dataframe X
X = adult_dt.drop(columns=['income'])

# Create the target dataframe Y
Y = adult_dt['income']

# Perform the train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

The random state of the splitting function is a parameter that controls the randomness of the data split. By setting a specific value (like 42), I ensure that every time I run the code, I get the same split of data into training and testing sets.

This is incredibly useful for reproducibility. It allows others to replicate my results precisely, ensuring consistency across experiments and making my work verifiable and reliable.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder

# Columns
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Numerical pipeline
numerical_pipeline = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Categorical pipeline
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)

# Example usage
# X_train_preprocessed = preprocessor.fit_transform(X_train)
# X_test_preprocessed = preprocessor.transform(X_test)


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [5]:
from sklearn.ensemble import RandomForestClassifier

# Model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Example usage
# model_pipeline.fit(X_train, Y_train)
# predictions = model_pipeline.predict(X_test)


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [6]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Define the scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}

# Perform cross-validation
cv_results = cross_validate(model_pipeline, X_train, Y_train, cv=5, scoring=scoring, return_train_score=True)

# Print the results
print("Training Scores:")
for metric in scoring:
    print(f"{metric}: {cv_results[f'train_{metric}']}")

print("\nValidation Scores:")
for metric in scoring:
    print(f"{metric}: {cv_results[f'test_{metric}']}")


c:\Users\huiho\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\huiho\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training Scores:
neg_log_loss: [-0.08123469 -0.08144442 -0.08159964 -0.08190836 -0.08146028]
roc_auc: [1. 1. 1. 1. 1.]
accuracy: [1.         1.         0.99994516 1.         0.99994516]
balanced_accuracy: [1.         1.         0.99988693 1.         0.99988693]

Validation Scores:
neg_log_loss: [-0.33253014 -0.37454446 -0.36914766 -0.37843457 -0.36400961]
roc_auc: [0.90695108 0.90226427 0.90092948 0.90598011 0.9040762 ]
accuracy: [0.8499671  0.84952841 0.85212813 0.85761299 0.8571742 ]
balanced_accuracy: [0.77497173 0.77038024 0.77440589 0.77802593 0.77712095]


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [7]:
# Create a DataFrame from cross-validation results
cv_results_df = pd.DataFrame(cv_results)

# Sort by negative log loss of the test set
sorted_cv_results_df = cv_results_df.sort_values(by='test_neg_log_loss')

print(sorted_cv_results_df)


    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
3  12.588922    0.225141          -0.378435           -0.081908      0.905980   
1  12.891906    0.226847          -0.374544           -0.081444      0.902264   
2  12.447942    0.221463          -0.369148           -0.081600      0.900929   
4  12.506435    0.220146          -0.364010           -0.081460      0.904076   
0  14.168888    0.251661          -0.332530           -0.081235      0.906951   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
3            1.0       0.857613        1.000000                0.778026   
1            1.0       0.849528        1.000000                0.770380   
2            1.0       0.852128        0.999945                0.774406   
4            1.0       0.857174        0.999945                0.777121   
0            1.0       0.849967        1.000000                0.774972   

   train_balanced_accuracy  
3                 1.000000  
1   

Calculate the mean of each metric. 

In [8]:
# Calculate the mean of each metric
mean_scores = cv_results_df.mean()

print(mean_scores)

fit_time                   12.920819
score_time                  0.229052
test_neg_log_loss          -0.363733
train_neg_log_loss         -0.081529
test_roc_auc                0.904040
train_roc_auc               1.000000
test_accuracy               0.853282
train_accuracy              0.999978
test_balanced_accuracy      0.774981
train_balanced_accuracy     0.999955
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [11]:
# Fit the model pipeline
model_pipeline.fit(X_train, Y_train)

# Generate predictions and prediction probabilities
Y_pred = model_pipeline.predict(X_test)
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculate the performance metrics
metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

print(metrics)



{'neg_log_loss': -0.3745366745621568, 'roc_auc': 0.9005347556384113, 'accuracy': 0.8550516941345071, 'balanced_accuracy': 0.7761391454459978}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Recoding the target variable income immediately during data loading is convenient because it ensures the transformation is handled efficiently and consistently, right at the start, which avoids additional steps and potential errors later. This keeps the codebase clean and easy to read, as the transformation is done in one concise operation. Using the assign method with a lambda function makes the process straightforward and maintains the binary format needed for classification tasks.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.